<a href="https://colab.research.google.com/github/iam-fern/Research/blob/main/ex_sampling_100rep_n50_x6_sas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import statistics as stats
import scipy.stats as ss
import scipy.stats as st
from sklearn.linear_model import LinearRegression
from scipy import stats
import scipy.stats as stats
import time
import statsmodels.api as sm

from statsmodels.formula.api import ols

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
df = pd.read_csv('https://github.com/iam-fern/Research/raw/main/sampling_100rep_n50_x6_sas.csv', index_col=['rep', 'index'])
df

x1         x2  ...         y_hat         e_hat
rep  index                        ...                            
0.0  0.0    15.664463  33.549879  ...  1.489033e+07 -2.368560e+07
     1.0    43.094629  10.742496  ... -7.606243e+06 -4.729199e+09
     2.0    14.309655  12.277679  ... -6.766850e+07  7.603971e+07
     3.0    47.432676  42.396775  ...  1.312166e+08  4.768706e+09
     4.0    30.240323  37.426796  ...  6.619081e+07 -5.704736e+08
...               ...        ...  ...           ...           ...
99.0 45.0   11.711733  12.097388  ... -7.631987e+07  8.222427e+07
     46.0   47.316663  49.274670  ...  1.660624e+08 -6.670209e+09
     47.0   25.796852  15.997932  ... -2.560690e+07  1.883681e+08
     48.0   10.862142  26.158495  ... -2.216073e+07  2.153085e+07
     49.0   31.174159  10.584933  ... -4.016756e+07 -9.165646e+08

[5000 rows x 7 columns]

In [3]:
# ลบข้อมูลที่มี missing values
table = df.dropna()
table

x1         x2  ...         y_hat         e_hat
rep  index                        ...                            
0.0  0.0    15.664463  33.549879  ...  1.489033e+07 -2.368560e+07
     1.0    43.094629  10.742496  ... -7.606243e+06 -4.729199e+09
     2.0    14.309655  12.277679  ... -6.766850e+07  7.603971e+07
     3.0    47.432676  42.396775  ...  1.312166e+08  4.768706e+09
     4.0    30.240323  37.426796  ...  6.619081e+07 -5.704736e+08
...               ...        ...  ...           ...           ...
99.0 45.0   11.711733  12.097388  ... -7.631987e+07  8.222427e+07
     46.0   47.316663  49.274670  ...  1.660624e+08 -6.670209e+09
     47.0   25.796852  15.997932  ... -2.560690e+07  1.883681e+08
     48.0   10.862142  26.158495  ... -2.216073e+07  2.153085e+07
     49.0   31.174159  10.584933  ... -4.016756e+07 -9.165646e+08

[5000 rows x 7 columns]

In [4]:
table.reset_index(inplace=True)
table

,rep,index,x1,x2,x3,e,Y,y_hat,e_hat
0,0.0,0.0,15.664463,33.549879,21.577989,-8.795483e+06,-8.795266e+06,1.489033e+07,-2.368560e+07
1,0.0,1.0,43.094629,10.742496,21.772447,-4.736806e+09,-4.736806e+09,-7.606243e+06,-4.729199e+09
2,0.0,2.0,14.309655,12.277679,38.074888,8.371010e+06,8.371208e+06,-6.766850e+07,7.603971e+07
3,0.0,3.0,47.432676,42.396775,21.611796,4.899922e+09,4.899923e+09,1.312166e+08,4.768706e+09
4,0.0,4.0,30.240323,37.426796,18.361415,-5.042831e+08,-5.042828e+08,6.619081e+07,-5.704736e+08
...,...,...,...,...,...,...,...,...,...
4995,99.0,45.0,11.711733,12.097388,34.847377,5.904227e+06,5.904396e+06,-7.631987e+07,8.222427e+07
4996,99.0,46.0,47.316663,49.274670,39.398031,-6.504147e+09,-6.504146e+09,1.660624e+08,-6.670209e+09
4997,99.0,47.0,25.796852,15.997932,32.989470,1.627609e+08,1.627612e+08,-2.560690e+07,1.883681e+08
4998,99.0,48.0,10.862142,26.158495,33.650546,-6.300507e+05,-6.298772e+05,-2.216073e+07,2.153085e+07


In [5]:
table.set_index(['rep', 'index'], inplace=True)

In [6]:
table

x1         x2  ...         y_hat         e_hat
rep  index                        ...                            
0.0  0.0    15.664463  33.549879  ...  1.489033e+07 -2.368560e+07
     1.0    43.094629  10.742496  ... -7.606243e+06 -4.729199e+09
     2.0    14.309655  12.277679  ... -6.766850e+07  7.603971e+07
     3.0    47.432676  42.396775  ...  1.312166e+08  4.768706e+09
     4.0    30.240323  37.426796  ...  6.619081e+07 -5.704736e+08
...               ...        ...  ...           ...           ...
99.0 45.0   11.711733  12.097388  ... -7.631987e+07  8.222427e+07
     46.0   47.316663  49.274670  ...  1.660624e+08 -6.670209e+09
     47.0   25.796852  15.997932  ... -2.560690e+07  1.883681e+08
     48.0   10.862142  26.158495  ... -2.216073e+07  2.153085e+07
     49.0   31.174159  10.584933  ... -4.016756e+07 -9.165646e+08

[5000 rows x 7 columns]

In [8]:
def get_result(run_replicate, percent_data):
    new_table = table.loc[run_replicate]

    # เรียง data จากน้อยไปมากตาม x1
    new_table_sort = new_table.sort_values('x1')

    new_table_sort.reset_index(inplace=True)

    new_table_sort.drop(['index', 'y_hat', 'e_hat'], axis=1, inplace=True)


    # แบ่งข้อมูลกลุ่มที่ 1
    table1 = new_table_sort.iloc[:int(percent_data*new_table_sort.shape[0]) , :]
    y = table1['Y']
    x = table1[['x1', 'x2', 'x3']]
    x = sm.add_constant(x)
    model1 = sm.OLS(y, x).fit()

    # หาค่า df กลุ่ม 1
    DF1 = model1.df_resid
    # หาค่า SSE1 : Residual sum of squares
    SSE1 = model1.ssr
    # หาค่า MSE1
    MSE1 = model1.mse_resid


    # แบ่งข้อมูลกลุ่มที่ 2
    table2 = new_table_sort.iloc[int((1-percent_data)*new_table_sort.shape[0]): , :]

    y = table2['Y']
    x = table2[['x1', 'x2', 'x3']]
    x = sm.add_constant(x)
    model2 = sm.OLS(y, x).fit()
    #print(model.summary())

    # หาค่า df กลุ่ม 2
    DF2 = model2.df_resid
    # หาค่า SSE2 : : Residual sum of squares
    SSE2 = model2.ssr
    # หาค่า MSE2
    MSE2 = model2.mse_resid

    # ค่าสถิติทดสอบ
    GQ_test = MSE2 / MSE1


    df2 = DF2
    df1 = DF1

    # คำนวณค่า p-value
    # sf(x, dfn, dfd, loc=0, scale=1) : Survival function (also defined as 1 - cdf, but sf is sometimes more accurate)
    two_tailed = 2.0*(1.0-(ss.f.cdf(GQ_test, df2, df1)))

    # pdf(x, dfn, dfd, loc=0, scale=1) : Probability density function
    p_greater = ss.f.cdf(GQ_test, df2, df1)
    
    return df2, df1, SSE2, SSE1, MSE2, MSE1, GQ_test, two_tailed, p_greater
    

    # เปรียบเทียบค่า p-value กับค่า alpha
    # Homoscedasticity : 0
    # Heteroscedasticity : 1
    #if p_value < alpha:
        #result_test='Reject005=0 : Heteroscedasticity '
        #result_test01=1
    #else:
        #result_test='Reject005=1 : Homoscedasticity '
        #result_test01=0
    
    #return df1, df2, SSE1, SSE2, MSE1, MSE2, GQ_test, result_test, p_value, result_test01


# สร้าง list ของ replicates
list_replicate=list({i[0] for i in table.index})
list_replicate.sort()

# สร้าง list เพื่อเก็บค่า ทุกรอบที่ run
list_df1=[]
list_df2=[]
list_SSE1=[]
list_SSE2=[]
list_MSE1=[]
list_MSE2=[]
list_GQ_test=[]
#list_result_test=[]
list_two_tailed=[]
list_p_greater=[]
#list_result_test01=[]


# กำหนด % การแบ่งข้อมูล
percent_data=0.4

# ใช้ for loop สำหรับการ run ทุก replicates
for run_replicate in list_replicate:
    #DF1, DF2, SSE1, SSE2, M_SSE1, M_SSE2, GQ_test, result_test=get_result(run_replicate, percent_data)
    df1, df2, SSE1, SSE2, MSE1, MSE2, GQ_test, two_tailed, p_greater=get_result(run_replicate, percent_data)
    list_df1.append(df1)
    list_SSE1.append(SSE1)
    list_df2.append(df2)
    list_SSE2.append(SSE2)
    list_MSE1.append(MSE1)
    list_MSE2.append(MSE2)
    list_GQ_test.append(GQ_test)
    #list_result_test.append(result_test)
    list_two_tailed.append(two_tailed)
    list_p_greater.append(p_greater)
    #list_result_test01.append(result_test01)

In [9]:
# สร้างตารางผลลัพธ์
alpha = 0.05
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
result=pd.DataFrame({"Replicate":list_replicate, "df1":list_df1, "df2":list_df2, "SSE1":list_SSE1,
                     "SSE2":list_SSE2, "MSE1":list_MSE1, "MSE2":list_MSE2,
                     "GQ_test":list_GQ_test, "two_tailed":list_two_tailed, "greater":list_p_greater})
result.set_index(['Replicate'], inplace=True)
result=result[(result["two_tailed"]<=1) & (result["greater"]<=1)]
result["Result_test"]=result["two_tailed"].apply(lambda p_value: 'Reject005=0 : Heteroscedasticity' 
                                                                  if p_value < alpha else 'Reject005=1 : Homoscedasticity')
#two_tailed_new_show=two_tailed_new.drop(columns="greater")

result.astype(str)

,df1,df2,SSE1,SSE2,MSE1,MSE2,GQ_test,two_tailed,greater,Result_test
Replicate,,,,,,,,,,
0.0,16.0,16.0,1.0541977904073694e+21,9.955831550509599e+17,6.588736190046059e+19,6.222394719068499e+16,1058.87467567027,2.220446049250313e-16,0.9999999999999999,Reject005=0 : Heteroscedasticity
1.0,16.0,16.0,9.527921038534692e+20,1.0449166458152242e+17,5.9549506490841825e+19,6530729036345151.0,9118.355111570827,2.220446049250313e-16,0.9999999999999999,Reject005=0 : Heteroscedasticity
2.0,16.0,16.0,6.119402862968427e+20,3.682085197093165e+17,3.824626789355267e+19,2.301303248183228e+16,1661.9395085696037,2.220446049250313e-16,0.9999999999999999,Reject005=0 : Heteroscedasticity
3.0,16.0,16.0,1.326001813152203e+21,4.705324856460085e+16,8.287511332201269e+19,2940828035287553.0,28180.877061690957,2.220446049250313e-16,0.9999999999999999,Reject005=0 : Heteroscedasticity
4.0,16.0,16.0,5.2770194461412164e+20,1.2288753680623414e+18,3.2981371538382602e+19,7.680471050389634e+16,429.4186036507415,2.220446049250313e-16,0.9999999999999999,Reject005=0 : Heteroscedasticity
5.0,16.0,16.0,1.4138691948143426e+21,1.4546351620367008e+17,8.836682467589641e+19,9091469762729380.0,9719.751259379156,2.220446049250313e-16,0.9999999999999999,Reject005=0 : Heteroscedasticity
6.0,16.0,16.0,8.547284212424629e+20,1.876388627001859e+17,5.342052632765393e+19,1.1727428918761618e+16,4555.17801026203,2.220446049250313e-16,0.9999999999999999,Reject005=0 : Heteroscedasticity
7.0,16.0,16.0,2.98718500802108e+20,4.8259871768267776e+17,1.866990630013175e+19,3.016241985516736e+16,618.9790603598823,2.220446049250313e-16,0.9999999999999999,Reject005=0 : Heteroscedasticity
8.0,16.0,16.0,9.287868376905824e+20,2.8245646001221896e+16,5.80491773556614e+19,1765352875076368.5,32882.47815788683,2.220446049250313e-16,0.9999999999999999,Reject005=0 : Heteroscedasticity


In [10]:
# สร้างตารางผลลัพธ์
alpha = 0.05
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
result=pd.DataFrame({"Replicate":list_replicate, "df1":list_df1, "df2":list_df2, "SSE1":list_SSE1,
                     "SSE2":list_SSE2, "MSE1":list_MSE1, "MSE2":list_MSE2,
                     "GQ_test":list_GQ_test, "two_tailed":list_two_tailed, "greater":list_p_greater})
result.set_index(['Replicate'], inplace=True)

result['two_tailed_new']=result[["two_tailed","greater"]].apply(lambda value: value[0] if value[0]<0.5 else 2*value[1], axis=1)
result["Result_test"]=result["two_tailed"].apply(lambda p_value: 'Reject005=0 : Heteroscedasticity' 
                                                                  if p_value < alpha else 'Reject005=1 : Homoscedasticity')
#two_tailed_new_show=two_tailed_new.drop(columns="greater")
result_show=result.drop(columns=["two_tailed", "greater"])
result_show.astype(str)

,df1,df2,SSE1,SSE2,MSE1,MSE2,GQ_test,two_tailed_new,Result_test
Replicate,,,,,,,,,
0.0,16.0,16.0,1.0541977904073694e+21,9.955831550509599e+17,6.588736190046059e+19,6.222394719068499e+16,1058.87467567027,2.220446049250313e-16,Reject005=0 : Heteroscedasticity
1.0,16.0,16.0,9.527921038534692e+20,1.0449166458152242e+17,5.9549506490841825e+19,6530729036345151.0,9118.355111570827,2.220446049250313e-16,Reject005=0 : Heteroscedasticity
2.0,16.0,16.0,6.119402862968427e+20,3.682085197093165e+17,3.824626789355267e+19,2.301303248183228e+16,1661.9395085696037,2.220446049250313e-16,Reject005=0 : Heteroscedasticity
3.0,16.0,16.0,1.326001813152203e+21,4.705324856460085e+16,8.287511332201269e+19,2940828035287553.0,28180.877061690957,2.220446049250313e-16,Reject005=0 : Heteroscedasticity
4.0,16.0,16.0,5.2770194461412164e+20,1.2288753680623414e+18,3.2981371538382602e+19,7.680471050389634e+16,429.4186036507415,2.220446049250313e-16,Reject005=0 : Heteroscedasticity
5.0,16.0,16.0,1.4138691948143426e+21,1.4546351620367008e+17,8.836682467589641e+19,9091469762729380.0,9719.751259379156,2.220446049250313e-16,Reject005=0 : Heteroscedasticity
6.0,16.0,16.0,8.547284212424629e+20,1.876388627001859e+17,5.342052632765393e+19,1.1727428918761618e+16,4555.17801026203,2.220446049250313e-16,Reject005=0 : Heteroscedasticity
7.0,16.0,16.0,2.98718500802108e+20,4.8259871768267776e+17,1.866990630013175e+19,3.016241985516736e+16,618.9790603598823,2.220446049250313e-16,Reject005=0 : Heteroscedasticity
8.0,16.0,16.0,9.287868376905824e+20,2.8245646001221896e+16,5.80491773556614e+19,1765352875076368.5,32882.47815788683,2.220446049250313e-16,Reject005=0 : Heteroscedasticity


In [11]:
result_show['Result_test'].value_counts()

Reject005=0 : Heteroscedasticity    100
Name: Result_test, dtype: int64